In [1]:
require 'countries'

true

# 80. コーパスの整形

文を単語列に変換する最も単純な方法は，空白文字で単語に区切ることである． ただ，この方法では文末のピリオドや括弧などの記号が単語に含まれてしまう． そこで，コーパスの各行のテキストを空白文字でトークンのリストに分割した後，各トークンに以下の処理を施し，単語から記号を除去せよ．

トークンの先頭と末尾に出現する次の文字を削除: .,!?;:()[]'"
空文字列となったトークンは削除
以上の処理を適用した後，トークンをスペースで連結してファイルに保存せよ．

In [ ]:
doc = []
pat = "[#{["\\", "\\.", "\\,", "\\!", "\\?", "\\;", "\\:", "\\(", "\\)", "\\[", "\\]", "\\'", "\""].join()}]"

File.foreach('../data/enwiki-20150112-400-r100-10576.txt').with_index do |line, index|
  p index if index % 50000 == 0
  sentence = line.chomp.split.map {|token|
    token.
      sub(/^#{pat}/, '').
      sub(/#{pat}$/, '').
      gsub(/#{pat}/, '') # ズル
    }.
    reject{|token| token.empty?}
  
  doc << sentence.join("\s") if sentence.size > 5
end

1

0
50000
100000
150000
200000


# 81. 複合語からなる国名への対処

英語では，複数の語の連接が意味を成すことがある．例えば，アメリカ合衆国は“United States”，イギリスは“United Kingdom”と表現されるが，“United”や“States”，“Kingdom”という単語だけでは，指し示している概念・実体が曖昧である．そこで，コーパス中に含まれる複合語を認識し，複合語を1語として扱うことで，複合語の意味を推定したい．しかしながら，複合語を正確に認定するのは大変むずかしいので，ここでは複合語からなる国名を認定したい．
インターネット上から国名リストを各自で入手し，80のコーパス中に出現する複合語の国名に関して，スペースをアンダーバーに置換せよ．例えば，“United States”は“United_States”，“Isle of Man”は“Isle_of_Man”になるはずである．

In [ ]:
original = ISO3166::Country.all.map{|country| country.name.sub(/\s\(.*?\)\s/, '')}.select{|name| name.match(/\s/)}
under_scored = original.map{|name| name.gsub(/\s/, '_')}

doc.each_with_index do |sentence, index|
  p index if index % 50000 == 0
  original.size.times do |i|
    sentence.gsub!(original[i-1], under_scored[i-1])
  end
end

1

# 82. 文脈の抽出

81で作成したコーパス中に出現するすべての単語tに関して，単語tと文脈語cのペアをタブ区切り形式ですべて書き出せ．ただし，文脈語の定義は次の通りとする．
- ある単語tの前後d単語を文脈語cとして抽出する（ただし，文脈語に単語tそのものは含まない）
- 単語tを選ぶ度に，文脈幅dは{1,2,3,4,5}の範囲でランダムに決める．

In [ ]:
def window
  (Random.rand * 4.9999).to_i + 1
end

output = open("../data/context.tsv", "w")

doc.each_with_index do |words, index|
  p index if index % 50000 == 0
  words = words.split(/\s|\n/)
  
  words.size.times.each do |i|
    w = window

    start_from = [0, i - w].max
    end_to = [words.size, i + w].min

    contexts = i+1 <= words.size ? words[start_from..i] + words[i + 1..end_to] : words[start_from..i]

    contexts.each do |context|
      output.puts "#{words[i]}\t#{context}"
    end
  end
end

output.close

1

In [ ]:
require 'daru'

In [ ]:
Daru::DataFrame.from_csv("../data/context.tsv") if false # つくれなかった